<p align="center">
    <img src="https://vectorseek.com/wp-content/uploads/2023/12/Mistral-AI-Icon-Logo-Vector.svg-.png" alt="Mistral.ai" width="40" style="margin-right: 20px;" align="center">
    <img src="https://upload.wikimedia.org/wikipedia/commons/2/2d/Tensorflow_logo.svg" alt="TensorFlow" width="35" align="center">
</p>
<h1 align="center">Estimation de la note d'un commentaire avec Mistral.ai, XGBoost et TensorFlow</h1>

---

Ce notebook est un outil d'exploration qui vise à étudier la possibilité d'utiliser les embeddings fournies par Mistral pour prédire la note associée à un commentaire. Pour accélérer le traitement des commentaires et de leurs embeddings, nous utiliserons la méthode des chunks. Cette approche consiste à diviser les données en plusieurs morceaux, ce qui permet de gérer les données de manière plus efficace et rapide.

Une fois les embeddings récupérées, nous utiliserons deux méthodes distinctes pour entraîner notre modèle de prédiction : 
- XGBoost 
- TensorFlow 

**XGBoost** est un algorithme de gradient boosting qui a fait ses preuves dans de nombreux domaines d'application, notamment pour la classification et la régression. <br>
<p align="center">
    <img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fdzone.com%2Fstorage%2Ftemp%2F13069527-boosting-algo.png&f=1&nofb=1&ipt=1ac6f470930257e0fb83997b0768a166f0c2b93a8ecd29dfc184c95efd1e786d&ipo=images" alt="XGBoost" width="500">
</p>

**TensorFlow** est une bibliothèque open source pour l'apprentissage automatique qui permet de créer des réseaux de neurones profonds et complexes.
<p align="center">
    <img src="https://s3.amazonaws.com/stackabuse/media/intro-to-neural-networks-scikit-learn-3.png" alt="neural network" width="300">
</p>
Dans un premier temps, nous préparerons les données en effectuant un prétraitement des commentaires et en récupérant les embeddings associées. Nous diviserons ensuite les données en un ensemble d'entraînement et un ensemble de test, ce qui nous permettra d'évaluer les performances de notre modèle. 

Nous entraînerons ensuite notre modèle à l'aide des deux méthodes mentionnées précédemment. Pour XGBoost, nous utiliserons l'algorithme de classification pour prédire la note associée à chaque commentaire. Pour TensorFlow, nous créerons un réseau de neurones profond avec plusieurs couches cachées et une couche de sortie avec cinq neurones, correspondant aux cinq notes possibles.

Enfin, nous évaluerons les performances de notre modèle sur l'ensemble de test en utilisant des métriques telles que la précision, le rappel et le score F1. Nous comparerons également les performances des deux méthodes pour déterminer celle qui convient le mieux à notre problème.

Dans l'ensemble, ce notebook vise à explorer la faisabilité d'utiliser les embeddings fournies par Mistral pour prédire la note associée à un commentaire. En utilisant deux méthodes distinctes, nous chercherons à déterminer la meilleure approche pour résoudre ce problème et à évaluer les performances de notre modèle.

---

# Installation et importation des packages

In [4]:
#%pip install -U xgboost scikit-learn mistralai tensorflow pandas numpy
from mistralai.client import MistralClient
import pandas as pd
import tensorflow as tf
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support

# Importation de la base de données

In [5]:
review = pd.read_csv("singapore_airlines_reviews.csv")
data = review.drop(columns=["published_date", "published_platform", "type", "title", "helpful_votes"])
data = data.head(2000)

# Définition de la méthode d'Embeddings

Mise en place de la fonction d'embeddings :

In [6]:
client = MistralClient(api_key="TVgCvsfUiqr5m8F1JSleadiEpHF5UdhL")

def get_embeddings_by_chunks(data, chunk_size):
    chunks = [data[x : x + chunk_size] for x in range(0, len(data), chunk_size)]
    embeddings_response = [
        client.embeddings(model="mistral-embed", input=c) for c in chunks
    ]
    return [d.embedding for e in embeddings_response for d in e.data]

Application de la fonction d'embeddings :

In [7]:
data["embeddings"] = get_embeddings_by_chunks(data["text"].tolist(), 10)
#np.array(data["embeddings"])

# Prépatation des données d'entrainement et de test

In [8]:
X_train, X_test, y_train, y_test = train_test_split(np.array(np.array(data["embeddings"]).tolist()), np.array(data["rating"]), test_size=0.2, random_state=42)

## XGBoost

In [9]:
# Entraînement du modèle

# Définir les paramètres du modèle XGBoost
params = {
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'num_class': 6,
    'max_depth': 3,
    'eta': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

# Convertir les données en formats DMatrix de XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Entraîner le modèle XGBoost
model = xgb.train(params, dtrain, num_boost_round=100)

# Évaluation du modèle

# Évaluer le modèle sur l'ensemble de test
y_pred = model.predict(dtest)
y_pred = np.argmax(y_pred, axis=1)
accuracy = np.mean(y_pred == y_test)
print("Précision sur l'ensemble de test : {:.2f}".format(accuracy))

precision, recall, fscore, support = precision_recall_fscore_support(y_test, y_pred)


Précision sur l'ensemble de test : 0.74


## Tensorflow

In [10]:
X_train, X_test, y_train, y_test = train_test_split(np.array(np.array(data["embeddings"]).tolist()), np.array(data["rating"]), test_size=0.2, random_state=42)

y_train = y_train - 1
y_test = y_test - 1

model = tf.keras.Sequential([
    tf.keras.layers.Dense(1024, activation='tanh', input_shape=(1024,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5, activation='softmax')
])

# Compiler le modèle Keras
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=["accuracy"])

# Entraîner le modèle Keras
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Évaluation du modèle

# Évaluer le modèle sur l'ensemble de test
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
mse = ((y_test - y_pred) ** 2).mean()
print("Erreur quadratique moyenne sur l'ensemble de test : {:.2f}".format(mse))

/Users/Jbastruz/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.4856 - loss: 1.3041 - val_accuracy: 0.7125 - val_loss: 0.7075
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.6685 - loss: 0.8329 - val_accuracy: 0.6938 - val_loss: 0.7180
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7140 - loss: 0.7584 - val_accuracy: 0.6938 - val_loss: 0.7507
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7157 - loss: 0.7743 - val_accuracy: 0.7625 - val_loss: 0.6312
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7302 - loss: 0.7186 - val_accuracy: 0.7500 - val_loss: 0.6856
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7345 - loss: 0.7517 - val_accuracy: 0.7375 - val_loss: 0.6657
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7372 - loss: 0.6854 - val_accuracy: 0.7312 - val_loss: 0.6324
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7507 - loss: 0.6260 - val_accuracy: 0.7437 - v

In [11]:
print(y_pred + 1)
print(y_test + 1)


[5 3 5 1 4 4 1 2 2 5 1 5 2 3 1 3 1 3 3 4 2 5 5 5 5 1 1 3 4 3 1 1 1 1 1 5 3
 5 5 5 5 5 5 3 5 4 1 4 5 5 5 1 4 1 1 5 1 5 1 3 5 5 5 1 5 5 3 3 2 1 3 3 5 1
 1 5 5 1 1 5 5 1 5 2 5 4 5 2 5 5 5 4 3 2 5 5 3 1 5 1 1 1 1 4 3 5 1 5 1 2 1
 5 4 2 3 5 2 1 3 5 4 5 5 5 3 5 1 1 1 2 4 4 1 1 1 1 2 5 5 5 1 3 5 1 5 5 1 1
 1 1 5 1 1 2 1 1 5 5 3 3 5 1 5 1 5 5 5 4 2 5 5 1 5 5 1 1 2 1 1 5 5 4 1 1 3
 1 4 4 1 1 2 5 5 5 5 4 5 1 1 3 5 5 5 1 4 2 5 5 1 2 5 1 2 1 3 5 4 4 1 5 1 1
 1 1 5 5 3 5 5 1 5 1 5 5 3 1 5 1 5 5 1 5 3 1 3 4 5 4 1 5 2 1 1 4 1 2 1 5 4
 2 2 5 2 5 1 5 1 3 5 1 3 1 1 5 1 5 1 5 4 1 5 1 1 5 5 4 1 5 5 5 1 3 4 5 5 1
 3 5 3 1 5 1 5 2 1 4 1 5 1 4 1 5 3 1 2 2 5 5 5 4 1 4 3 4 3 1 5 1 5 1 1 1 5
 5 5 3 1 2 1 5 1 5 1 1 1 5 2 5 5 3 4 1 4 3 1 2 5 4 1 1 5 3 5 5 1 4 5 2 1 1
 5 5 3 5 5 4 3 5 5 5 5 1 1 1 5 1 1 1 5 5 4 4 1 1 3 1 5 5 1 2]
[5 3 5 2 4 4 2 3 2 5 1 5 3 5 1 2 1 4 4 4 2 5 4 5 5 1 1 4 5 2 1 2 3 2 2 5 3
 4 5 5 5 5 5 1 5 4 1 3 5 5 5 1 5 1 1 5 1 4 1 4 5 5 5 1 5 5 2 4 2 1 4 2 5 1
 1 5 5 1 1 5 5 2 5 2 5 4 5 1 5 5 4 4 3

In [12]:
precision, recall, fscore, support = precision_recall_fscore_support(y_test+1, y_pred+1)

In [13]:
for i in range(len(precision)):
    print(f"Classe {i+1} : Précision = {precision[i]:.2f}, Rappel = {recall[i]:.2f}, Fscore = {fscore[i]:.2f}")

Classe 1 : Précision = 0.77, Rappel = 0.84, Fscore = 0.80
Classe 2 : Précision = 0.36, Rappel = 0.26, Fscore = 0.30
Classe 3 : Précision = 0.42, Rappel = 0.54, Fscore = 0.47
Classe 4 : Précision = 0.56, Rappel = 0.43, Fscore = 0.48
Classe 5 : Précision = 0.87, Rappel = 0.90, Fscore = 0.89
